# Wrangle-OpenStreetMap-Data

On the particular project, I am using data mungling techniques to assess the quality of OpenStreetMap’s (OSM) data for the mumbai city regarding their consistency and uniformity. The data wrangling takes place programmatically, using Python for the most of the process and SQL for items that need further attention.

The dataset describes the city of mumbai.Mumbai,India is the closest thing I have to a hometown in the India as I lived there for a good chunk of my childhood, so I was keen to take a look at it in this new, OpenStreetMap-based lens. The size of the dataset is 66 MB and can can be downloaded from here: https://mapzen.com/data/metro-extracts/metro/mumbai_india/

About the project

Scope

OpenStreetMap (OSM) is a collaborative project to create a free editable map of the world. The creation and growth of OSM have been motivated by restrictions on use or availability of map information across much of the world, and the advent of inexpensive portable satellite navigation devices.

On the specific project, I am using data from https://www.openstreetmap.org/node/16173235 and data mungling techniques, to assess the quality of their validity, accuracy, completeness, consistency and uniformity. The biggest part of the wrangling takes place programmatically using Python and then the dataset is entered into a SQL database for further examination of any remaining elements that need attention. Finally, I perform some basic exploration and express some ideas for additional improvements.

Skills demonstrated

Assessment of the quality of data for validity, accuracy, completeness, consistency and uniformity. Parsing and gathering data from popular file formats such as .xml and .csv. Processing data from very large files that cannot be cleaned with spreadsheet programs. Storing, querying, and aggregating data using SQL. Mumbai, India

https://www.openstreetmap.org/node/16173235 https://mapzen.com/data/metro-extracts/metro/mumbai_india/

Problems Encountered in the Map

Problems Encountered in the Map Once the location was decided, I downloaded the full extract of the region and ran Python code to investigate any issues with the data. The following problems were discovered:

Street Names: Incomplete ('hanuman raod ___') or incorrect names ('Zhopadpatti'), along with street abbreviations ('rd.' instead of 'Road')

Postal Codes: Inconsistent postal code formats ('500023' and '120045') and incorrect post codes ('123')

To tackle these issues, I had to create python scripts to clean each respective category of data. Auditing part is explained in Openstreetmap.ipynb notebook

I have created mumbai_sample file which is part of mumbai_india file and it can be used for various experiments before using those on main osm file, we can also get the idea about the format of osm file.

In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "mumbai_india.osm"  # Replace this with your osm file
SAMPLE_FILE = "mumbai_sample.osm"

k = 500 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')


Different types of tags mumbai_india file

In [2]:
import xml.etree.cElementTree as ET
import pprint
tags={}
def count_tags(sample_file):
    for event, elem in ET.iterparse(sample_file, events=("start",)):
        if elem.tag in tags.keys():
            tags[elem.tag] += 1
        else:
            tags[elem.tag] = 1
        #print tags
    return tags
tags = count_tags('mumbai_india.osm')
pprint.pprint(tags)

{'bounds': 1,
 'member': 13085,
 'nd': 2356083,
 'node': 2051208,
 'osm': 1,
 'relation': 3994,
 'tag': 391731,
 'way': 283963}


Tag types 

  "lower", for tags that contain only lowercase letters and are valid,
  
  "lower_colon", for otherwise valid tags with a colon in their names,
  
  "problemchars", for tags with problematic characters, 
  
  "other", for other tags that do not fall into the other three categories.

In [52]:
import xml.etree.cElementTree as ET
import pprint
import re
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
            if lower.search(element.attrib['k']):
              keys["lower"] += 1
            elif lower_colon.search(element.attrib['k']):
              keys["lower_colon"] += 1
            elif problemchars.search(element.attrib['k']):
              keys["problemchars"] += 1
            else:
              keys["other"] += 1
    return keys



def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys
keys = process_map('mumbai_india.osm')
pprint.pprint(keys)

{'lower': 375605, 'lower_colon': 15589, 'other': 530, 'problemchars': 7}


To find number of unique users in mumbai_india osm file

In [51]:
import xml.etree.cElementTree as ET
import pprint
import re
def get_user(element):
     if "uid" in element.attrib:
        return element.attrib["uid"]


def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
       users.add(get_user(element))
       users.discard(None)
    return users
users = process_map('mumbai_india.osm')
#pprint.pprint(users)
len(users)


1749

# Street Names
There are two things the auditing function needed to accomplish:
Replace any abbreviations of street types with the type completely spelled out.
  1. 'Veer Savarkar Rd' -> 'Veer Savarkar Road'
  
Replace incorrect or incomplete street names with the corrected/complete counterparts.

  2. 'jhopadpatti' -> 'Slums'

In order to discover problematic street names, I first had to use the regular expression (re) module to locate street types at the end of an address.

In [ ]:
OSMFILE = "mumbai_india.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


Afterwards, I matched them against a list of acceptable street types. If they weren't in the list of expected types, they would be added to a dictionary as keys, with the addresses that contain the problematic cases as the values.

Having this overview allowed me to determine what my auditing function needed to accomplish. I created a dictionaries for mapping/correcting purposes - 'mapping'. If my function came across a problematic street type, it would refer to that dictionaries for the corrected version to be replaced with.

An analysis of the XML data, along with outside research on Google Maps and OpenStreetMaps, was needed to identify the missing street types for the incomplete street names.


In [33]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "mumbai_india.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

#expected = ["Slums", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            
# UPDATE THIS VARIABLE
mapping = { "Jhopadpatti": "Slums",
            "Marg,": "Marg",
            "Mumbai,":"Mumbai",
            "ROad":"Road",
            "Rd":"Road",
            "Raod":"Road",
            "Rd.":"Road",
            "road":"Road"
            }




# UPDATE THIS VARIABLE


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        #if street_type not in expected:
        street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

def update_name(name, mapping):
    #print '\nSTART'
    #print name
    m = street_type_re.search(name)
    if m.group() in mapping.keys():        
        name=re.sub(street_type_re,mapping[m.group()],name)

    # YOUR CODE HERE

    return name

st_types = audit(OSMFILE)
#pprint.pprint(dict(st_types))

for st_type, ways in st_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        print name, "=>", better_name  

Universal Mobile,DVD and Travelling shop => Universal Mobile,DVD and Travelling shop
Vidhan Bhavan => Vidhan Bhavan
kurla => kurla
lalluabai compound => lalluabai compound
THAKKAR BAPPA COLONY => THAKKAR BAPPA COLONY
DATAR COLONY => DATAR COLONY
P.L. Lokhande Marg,MHADA COLONY => P.L. Lokhande Marg,MHADA COLONY
Thakurli Station Rd => Thakurli Station Road
Veer Savarkar Rd => Veer Savarkar Road
Bhandup Village Rd => Bhandup Village Road
Sambhaji Nagar Rd => Sambhaji Nagar Road
Pipeline Rd => Pipeline Road
Vasai Station Rd => Vasai Station Road
NL Rd => NL Road
Marol Village Rd => Marol Village Road
Vishnu Nagar Rd => Vishnu Nagar Road
Yashavant Nagar Rd => Yashavant Nagar Road
vishnu Nagar Rd => vishnu Nagar Road
Balraj Sahni Rd => Balraj Sahni Road
Wodehouse Rd => Wodehouse Road
Koparkhairane Village Rd => Koparkhairane Village Road
Datta Mandir Rd => Datta Mandir Road
Belpada Rd => Belpada Road
Goregaon Mulund Link Rd => Goregaon Mulund Link Road
Agrawal Rd => Agrawal Road
Carmichael 

Barave gaon road => Barave gaon Road
Sher-e-Punjab road => Sher-e-Punjab Road
90 feet road => 90 feet Road
M. G. cross road => M. G. cross Road
Mulshi road => Mulshi Road
l.u.gadkari road => l.u.gadkari Road
90ft road => 90ft Road
Manish Market service road => Manish Market service Road
kirol road => kirol Road
mohmmad road => mohmmad Road
Raghunath Vihar road => Raghunath Vihar Road
bharda wadi road => bharda wadi Road
off Ghodhbunder road => off Ghodhbunder Road
Vashi station road => Vashi station Road
NMMT road => NMMT Road
bhivandi-kalher road => bhivandi-kalher Road
Harbour Heights service road => Harbour Heights service Road
amrutnager road => amrutnager Road
st anthony church road => st anthony church Road
Juhu Tara road => Juhu Tara Road
market road => market Road
Father peter periera road => Father peter periera Road
road => Road
Khokhani Estate, Opp. Neelkanth Industrial Estate, Near Saibaba Mandir, Sativali.Palgarh: => Khokhani Estate, Opp. Neelkanth Industrial Estate, Near 


# Postal Codes

Postal codes of Mumbai all start with '40' and all are of 6 digits therefore for auditing postcodes it was important to ensure the postal codes are in the 6-digit format
To locate and extract the 6-digit zip code in cases where there are irrelevant or too much information - i.e. random white spaces or an additional strings after the zip code.
Example conversions:
1. 400076,India

To identify problems with postal codes, I used a function that would search through the XML data for every instance of a postal code, and add it to a set of unique postal codes if it satisfies above mentioned conditions.
The audit function can be found below.

In [3]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "mumbai_india.osm"
tree = ET.parse(OSMFILE)
root = tree.getroot()

postal_type_re = re.compile('^(4)(0)\d{4}$')



def audit_postal_code(postal_types,postal_value):
    m = postal_type_re.search(postal_value)
    if m:
        postal_type=m.group()
        postal_types[postal_type].add(postal_value)

def is_postal_code(elem):
    return (elem.attrib['k'] == "addr:postcode")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    postal_types=defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag in["node", "way", "relation"] :
            for tag in elem.iter("tag"):
                if is_postal_code(tag):
                    audit_postal_code(postal_types,tag.attrib['v'])
    osm_file.close()
    return postal_types


#audit(SAMPLE_FILE)
#audit(OSMFILE)
pprint.pprint(dict(audit(OSMFILE)))



{'400001': set(['400001']),
 '400002': set(['400002']),
 '400003': set(['400003']),
 '400004': set(['400004']),
 '400005': set(['400005']),
 '400007': set(['400007']),
 '400008': set(['400008']),
 '400009': set(['400009']),
 '400010': set(['400010']),
 '400011': set(['400011']),
 '400012': set(['400012']),
 '400013': set(['400013']),
 '400014': set(['400014']),
 '400015': set(['400015']),
 '400016': set(['400016']),
 '400017': set(['400017']),
 '400018': set(['400018']),
 '400019': set(['400019']),
 '400020': set(['400020']),
 '400021': set(['400021']),
 '400022': set(['400022']),
 '400023': set(['400023']),
 '400024': set(['400024']),
 '400025': set(['400025']),
 '400027': set(['400027']),
 '400028': set(['400028']),
 '400029': set(['400029']),
 '400030': set(['400030']),
 '400031': set(['400031']),
 '400033': set(['400033']),
 '400034': set(['400034']),
 '400035': set(['400035']),
 '400036': set(['400036']),
 '400037': set(['400037']),
 '400038': set(['400038']),
 '400039': set(['400

Code below shows the bad postal codes which are either not of mumbai or are in incorrect format:

In [34]:
import xml.etree.ElementTree as ET
import re

tree = ET.parse(OSMFILE)
root = tree.getroot()

counttotal = 0
count = 0
wp = []
regex = re.compile('^(4)(0)\d{4}$')
for i in tree.getiterator('tag'):
    k1 = i.get("k")
    if k1 == "addr:postcode":
        v1 = i.get("v")
        m1 = regex.match(v1)
        if not m1:
            counttotal = counttotal +1
            if len(v1) <> 6:
                v1 = v1.replace(" ","")
                v1 = v1.replace(",","")
                v1 = v1.replace("-","")
                v1 = v1.replace('"',"")
                v1 = v1.replace('55',"5")
                m2 = regex.match(v1)
                if not m2:
                    wp.append(v1)
                    count = count +1
            elif len(v1) == 6:
                wp.append(v1)
                count = count +1

print wp
print count
#print counttotal


['421501', '421503', '410206', '410210', '410210', '410210', '410210', '412108', '410701', '421202', '421202', '4000607', '40058', '421501', '63103', '421501', '421501', '410201', '410201', '410201', '4000082', '400059.', '410210', '110092', '123', '410209', '410209', '410209', '421302', '410206', '4000072', '410206', '421301', '421301', '4000072', '421306', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410208', '421501', '410206', '40076', '560023', '590006', '492006', '410210', '410206', '560092', '410206', '410206', '410206', '410206', '410206', '410206', '410206', '410206', '410206', '410221', '410102', '410206', '421601', '410206', '421005', '410206', '421501', '410221', '410206', '410206', '48147', '410218', '421201', '421302', '410206', '412108', '412108', '421501', '421501', '410206', '421311', '400076India', '400076India', '410210', '40081', '4

This function is use to update the postal codes by which all postal codes from bad postals mentioned above will be replaced by '000000'.

In [37]:
def update_postcode(postcode):
    search = re.search(r'^\D*(\d{6}).*',postcode)
    search1 = re.search('^(4)(0)\d{4}$',postcode)
    if search1:
        return search.group(1) 
    else :
        return '000000'
for i in tree.getiterator('tag'):
    k1 = i.get("k")
    if k1 == "addr:postcode":
        v1 = i.get("v")
        print  update_postcode(v1)
        

000000
000000
000000
400050
400050
400050
400050
400615
400614
000000
400005
000000
400020
400086
400601
400022
400606
400039
400005
400001
400039
400020
000000
400039
000000
400039
400607
400020
400069
400703
400610
400016
400071
400067
400005
400049
400072
400710
400093
400054
400052
400088
000000
400001
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400607
400607
400607
400607
400607
400607
400607
400607
400607

400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400016
400031
400012
400070
400070
400086
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400012
400093
400101
400077
400086
400059
400022
400101
400022
400052
400099
400093
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400076
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400950
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400016
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050

# Parsing the XML File and Writing the Data to a CSV

In order to begin moving our data towards a SQL database for analysis, I first had to parse the XML file and transform it from a document format to a tabular format. I was then able to write the data across multiple (5) .csv files, allowing us to easily import it to a SQL database. It is in this script where the data cleaning functions are applied.

Below is the data. In this code portion of the shape_element function, handles the bulk of the format shaping and utilizes the audit functions I displayed above to clean the data as well. This portion focuses on shaping way_tags and nodes_tags from the XML file into a Python dictionary.Process_map function is used to write data into csv file.In shape_element function update_postcode and update_name functions are called.

In [38]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

OSM_PATH = "mumbai_india.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements
  
    # YOUR CODE HERE
    
    if element.tag == 'node':
        for i in NODE_FIELDS:
            node_attribs[i] = element.attrib[i]
        for tag in element.iter("tag"):
            tag_dict= {}
            # Calling the cleaning function
            # replace name with the value of the attribute
            if tag.attrib['k'] == 'addr:street':
                tag.attrib['v'] = update_name(tag.attrib['v'], mapping)
            if tag.attrib["k"] == 'addr:postcode':
                tag.attrib["v"] = update_postcode(tag.attrib["v"])
            
            tag_dict['id'] = node_attribs['id']
            key = tag.attrib['k']
            
            if re.search(PROBLEMCHARS, tag.attrib['k']):
                pass
            if re.search(LOWER_COLON, tag.attrib['k']):
                pass
            if ':' in tag.attrib['k']:
                type = key[: key.index(':')]
                key = key[key.index(':')+1 :]   
            else:
                type = 'regular'   
            
            tag_dict['key'] = key
            tag_dict['value'] = tag.attrib['v']
            tag_dict['type'] = type
            tags.append(tag_dict)
            
            #pprint.pprint(dict(tag_dict))
            
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        for way in WAY_FIELDS:
            way_attribs[way] = element.attrib[way]
        for tag in element.iter("tag"):
            tag_dict1= {}

            if tag.attrib["k"] == 'addr:postcode':
                tag.attrib["v"] = update_postcode(tag.attrib["v"])
            tag_dict1['id'] = way_attribs['id']
            key = tag.attrib['k']
            
            if re.search(PROBLEMCHARS, tag.attrib['k']):
                pass
            if re.search(LOWER_COLON, tag.attrib['k']):
                pass
            if ':' in tag.attrib['k']:
                type = key[: key.index(':')]
                key = key[key.index(':')+1 :]
            else:
                type = 'regular' 
            tag_dict1['key'] = key
            tag_dict1['value'] = tag.attrib['v']
            tag_dict1['type'] = type
            tags.append(tag_dict1) 
            
        i= 0
        for tag in element.iter("nd"):
            way_dict = {}
            way_dict["id"] = way_attribs["id"]
            way_dict["node_id"] = tag.attrib["ref"]
            way_dict["position"] = i
            way_nodes.append(way_dict)
            i +=1    
        
        # pprint.pprint(dict(way_dict)) 
            
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'wb') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'wb') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'wb') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'wb') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'wb') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)


# Importing CSV Files to SQL Database

With the CSV files written, I was able to begin the process of importing the data into a SQL database. In my case, I used SQLite 3 and accessed it through notebook to first create database named a Mumbai_india,then add all data from csv files to database.

There are two steps I needed to accomplish to complete this task:

1. Create five tables on sqlite based on a schema that matches their respective .csv files
2. Import each .csv file into the appropriate table

In [40]:
import csv, sqlite3
from pprint import pprint
sql_file="Mumbai_india.db"
con = sqlite3.connect(sql_file)
cur = con.cursor()

cur.execute('''DROP TABLE IF EXISTS nodes;''')
con.commit()


cur.execute("CREATE TABLE nodes(id INTEGER, lat REAL, lon REAL, user TEXT, uid INTEGER, version TEXT, changeset INTEGER, timestamp DATE);") # use your column names here
con.commit()
with open('nodes.csv','rb') as thr: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(thr) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"),i['lat'].decode("utf-8"),i['lon'].decode("utf-8"),i['user'].decode("utf-8"),i['uid'].decode("utf-8"),i['version'].decode("utf-8"),i['changeset'].decode("utf-8"),i['timestamp'].decode("utf-8")) for i in dr]

cur.executemany("INSERT INTO nodes (id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?,?,?);", to_db)
con.commit()
#con.close()


cur.execute('''DROP TABLE IF EXISTS nodes_tags;''')
con.commit()

cur.execute("CREATE TABLE Nodes_tags (id INTEGER, key TEXT, value TEXT, type TEXT);") # use your column names here
con.commit()
with open('nodes_tags.csv','rb') as thr: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(thr) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"),i['key'].decode("utf-8"),i['value'].decode("utf-8"),i['type'].decode("utf-8")) for i in dr]

cur.executemany("INSERT INTO Nodes_tags (id, key, value, type) VALUES (?,?,?,?);", to_db)
con.commit()

cur.execute('''DROP TABLE IF EXISTS ways;''')
con.commit()


cur.execute("CREATE TABLE ways (id INTEGER, user TEXT, uid INTEGER, version TEXT, changeset INTEGER, timestamp DATE);") # use your column names here
con.commit()
with open('ways.csv','rb') as thr: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(thr) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"),i['user'].decode("utf-8"),i['uid'].decode("utf-8"),i['version'].decode("utf-8"),i['changeset'].decode("utf-8"),i['timestamp'].decode("utf-8")) for i in dr]

cur.executemany("INSERT INTO ways (id , user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?);", to_db)
con.commit()


cur.execute('''DROP TABLE IF EXISTS ways_tags;''')
con.commit()


cur.execute("CREATE TABLE ways_tags (id INTEGER, key TEXT, value TEXT, type TEXT);") # use your column names here
con.commit()
with open('ways_tags.csv','rb') as thr: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(thr) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"),i['key'].decode("utf-8"),i['value'].decode("utf-8"),i['type'].decode("utf-8")) for i in dr]


cur.executemany("INSERT INTO ways_tags (id, key, value, type) VALUES (?,?,?,?);", to_db)
con.commit()



cur.execute('''DROP TABLE IF EXISTS ways_nodes;''')
con.commit()

cur.execute("CREATE TABLE ways_nodes (id INTEGER, node_id INTEGER, position INTEGER);") # use your column names here
con.commit()
with open('ways_nodes.csv','rb') as thr: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(thr) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"),i['node_id'].decode("utf-8"),i['position'].decode("utf-8")) for i in dr]
    
cur.executemany("INSERT INTO ways_nodes (id, node_id, position) VALUES (?,?,?);", to_db)
con.commit()


# Data Overview
In this section, I'll execute a number of SQL queries in order to analyze the dataset.

Users with most numbers of contributions.

In [41]:
sql_file="mumbai_india.db"
con = sqlite3.connect(sql_file)
cur = con.cursor()
cur.execute('SELECT user,count(*) FROM nodes GROUP BY user ORDER BY count(*) DESC LIMIT 5')
all_rows=cur.fetchall()
pprint(all_rows)


con.close()

[(u'parambyte', 69327),
 (u'PlaneMad', 68363),
 (u'anushap', 62553),
 (u'Ashok09', 62208),
 (u'Narsimulu', 55611)]


In [42]:
sql_file="mumbai_india.db"
con = sqlite3.connect(sql_file)
cur = con.cursor()
cur.execute('SELECT key,count(*) FROM ways_tags GROUP BY key ORDER BY count(*) DESC LIMIT 5')
all_rows=cur.fetchall()
pprint(all_rows)


con.close()

[(u'building', 223631),
 (u'highway', 40437),
 (u'name', 11741),
 (u'oneway', 4466),
 (u'source', 4142)]


Total number of unique users.

In [43]:
sql_file="mumbai_india.db"
con = sqlite3.connect(sql_file)
cur = con.cursor()
cur.execute('SELECT COUNT(DISTINCT(e.uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e')
all_rows=cur.fetchall()
pprint(all_rows)


con.close()

[(1739,)]


Total number of users who contributed for less than 5 times.

In [44]:
sql_file="mumbai_india.db"
con = sqlite3.connect(sql_file)
cur = con.cursor()
cur.execute('SELECT COUNT(*) FROM (SELECT e.user, COUNT(*) as num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) as e GROUP BY e.user HAVING num<=5)  u')
all_rows=cur.fetchall()
pprint(all_rows)


con.close()

[(910,)]


Different types of cuisines available in mumbai and number of restaurants it is available in.

In [45]:
sql_file="mumbai_india.db"
con = sqlite3.connect(sql_file)
cur = con.cursor()
cur.execute('SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value="restaurant") as i ON nodes_tags.id=i.id WHERE nodes_tags.key="cuisine" GROUP BY nodes_tags.value ORDER BY num DESC')
all_rows=cur.fetchall()
pprint(all_rows)


con.close()

[(u'indian', 66),
 (u'regional', 21),
 (u'pizza', 14),
 (u'vegetarian', 13),
 (u'chinese', 12),
 (u'italian', 11),
 (u'burger', 5),
 (u'international', 5),
 (u'seafood', 4),
 (u'asian', 3),
 (u'South_Indian', 2),
 (u'all_types_of_food', 2),
 (u'lebanese', 2),
 (u'Indian,_Chinese_etc', 1),
 (u'Seafood', 1),
 (u'Vegetarian_Restaurant', 1),
 (u'american', 1),
 (u'cafe', 1),
 (u'chicken;fish;indian', 1),
 (u'chicken;kebab;indian', 1),
 (u'chicken_,fish,cafe', 1),
 (u'fast_food', 1),
 (u'grill;coffee_shop;asian;noodles;fish_and_chips;diner;chicken;italian_pizza;indian;curry;fish;french;friture;chinese;barbecue',
  1),
 (u'indian;south_indian', 1),
 (u'indian_aagri', 1),
 (u'italian_pizza;pizza', 1),
 (u'lebanese,_chinese,_indian', 1),
 (u'local', 1),
 (u'mediterranean', 1),
 (u'only_vegiterian', 1),
 (u'oriental', 1),
 (u'persian', 1),
 (u'sad_food', 1),
 (u'south Indian; Punjabi; agari; malwani; Chinese', 1),
 (u'south_indian', 1),
 (u'south_indian,_chinese', 1),
 (u'spanish', 1),
 (u'swee

Different religions and number of places where they are workshipped.

In [46]:
sql_file="mumbai_india.db"
con = sqlite3.connect(sql_file)
cur = con.cursor()
cur.execute('SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags  JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value="place_of_worship") as i ON nodes_tags.id=i.id WHERE nodes_tags.key="religion" GROUP BY nodes_tags.value ORDER BY num DESC')
all_rows=cur.fetchall()
pprint(all_rows)


con.close()

[(u'hindu', 125),
 (u'muslim', 71),
 (u'christian', 34),
 (u'buddhist', 13),
 (u'jain', 6),
 (u'sikh', 4),
 (u'zoroastrian', 2),
 (u'jewish', 1)]


Different leisures

In [47]:
sql_file="mumbai_india.db"
con = sqlite3.connect(sql_file)
cur = con.cursor()
cur.execute('SELECT nodes_tags.value, count(*) as num FROM nodes_tags  WHERE nodes_tags.key="leisure" GROUP BY nodes_tags.value ORDER BY num DESC')
all_rows=cur.fetchall()
pprint(all_rows)


con.close()

[(u'park', 63),
 (u'playground', 20),
 (u'sports_centre', 15),
 (u'garden', 10),
 (u'fitness_centre', 8),
 (u'pitch', 6),
 (u'swimming_pool', 4),
 (u'aquarium', 1),
 (u'fitness_station', 1),
 (u'golf_course', 1),
 (u'stadium', 1)]


Names of few corrected streets.

In [48]:
sql_file="mumbai_india.db"
con = sqlite3.connect(sql_file)
cur = con.cursor()
cur.execute('SELECT value, count(*) as num FROM nodes_tags WHERE key="street" GROUP BY value ORDER BY num DESC LIMIT 10')

all_rows=cur.fetchall()
pprint(all_rows)


con.close()

[(u'Hanuman Road', 78),
 (u'Yashavant Nagar Road', 29),
 (u'Hiranandani Estate', 24),
 (u'P.L. Lokhande Marg', 24),
 (u'New Link Road, Andheri West', 21),
 (u'LBS Marg', 18),
 (u'Road Number 3', 18),
 (u'Thane Ghodbunder Road', 18),
 (u'Eastern Express Highway', 14),
 (u'GD Somani Road', 13)]


Cities surrounding mumbai and number of nodes in those cities.

In [49]:
sql_file="mumbai_india.db"
con = sqlite3.connect(sql_file)
cur = con.cursor()
cur.execute('SELECT tags.value, COUNT(*) as count FROM (SELECT * FROM nodes_tags UNION ALL  SELECT * FROM ways_tags) tags WHERE tags.key LIKE "%city" GROUP BY tags.value ORDER BY count DESC LIMIT 10')

all_rows=cur.fetchall()
pprint(all_rows)


con.close()

[(u'Mumbai', 607),
 (u'Bandra, Mumbai', 566),
 (u'mumbai', 187),
 (u'Virar West', 91),
 (u'Mulund (West)', 79),
 (u'Navi Mumbai', 70),
 (u'MUMBAI', 68),
 (u'Mulund (East)', 62),
 (u'Thane', 49),
 (u'Kharghar', 43)]


Different postcodes in ways_tags after using update postcode function.

In [50]:
sql_file="mumbai_india.db"
con = sqlite3.connect(sql_file)
cur = con.cursor()
cur.execute('SELECT value, COUNT(*) as count FROM ways_tags WHERE key="postcode" GROUP BY value ORDER BY count')

all_rows=cur.fetchall()
pprint(all_rows)


con.close()

[(u'400007', 1),
 (u'400010', 1),
 (u'400011', 1),
 (u'400034', 1),
 (u'400035', 1),
 (u'400038', 1),
 (u'400047', 1),
 (u'400059', 1),
 (u'400060', 1),
 (u'400066', 1),
 (u'400082', 1),
 (u'400095', 1),
 (u'400096', 1),
 (u'400602', 1),
 (u'400605', 1),
 (u'400703', 1),
 (u'400709', 1),
 (u'400710', 1),
 (u'400950', 1),
 (u'401104', 1),
 (u'401105', 1),
 (u'401209', 1),
 (u'402205', 1),
 (u'400015', 2),
 (u'400020', 2),
 (u'400025', 2),
 (u'400030', 2),
 (u'400054', 2),
 (u'400055', 2),
 (u'400068', 2),
 (u'400069', 2),
 (u'400079', 2),
 (u'400091', 2),
 (u'400098', 2),
 (u'400103', 2),
 (u'400612', 2),
 (u'400701', 2),
 (u'401101', 2),
 (u'400008', 3),
 (u'400024', 3),
 (u'400031', 3),
 (u'400037', 3),
 (u'400042', 3),
 (u'400056', 3),
 (u'400063', 3),
 (u'400092', 3),
 (u'400101', 3),
 (u'400102', 3),
 (u'400708', 3),
 (u'401202', 3),
 (u'400052', 4),
 (u'400077', 4),
 (u'400104', 4),
 (u'400606', 4),
 (u'400705', 4),
 (u'401201', 4),
 (u'400016', 5),
 (u'400022', 5),
 (u'400049', 5

# Potential Additional Improvements

There are several areas of improvement of the project in the future. The first one is on the completeness of the data. All the above analysis is based on a dataset that reflects a big part of mumbai but not only mumbai. The reason for this is the lack of a way to download a dataset for the entire mumbai without including parts of the neighboring cities. The analyst has to either select a part of the island/city or select a wider area that includes parts of thane and ratnagiri. Also, because of relations between nodes, ways, and relations, the downloaded data expand much further than the actual selection.

As a future improvement, I would download a wider selection or the metro extract from MapZen and filter the non-mumbai nodes and their references. The initial filtering could take place by introducing some latitude/longitude limits in the code to sort out most of the "non-m" nodes.

The second area with room for future improvement is the exploratory analysis of the dataset. Just to mention some of the explorings that could take place:

1.Distribution of commits per contributor.

2.Plotting of element creation per type, per day.

3.Distribution of distance between different types of amenities

4.Popular franchises in the country (fast food, conventional stores, etc.)

5.Selection of a bank based on the average distance you have to walk for an ATM.

6.Which area has the biggest parks and recreation spaces.

The scope of the current project was the wrangling of the dataset, so all the above have been left for future improvement.

Increasing Submissions

Going through this dataset, my concerns were less with the cleanliness of the data - as I found it surprisingly clean - and more with the lack of data. This part of mumbai is too big to have as little information as it does. I think OpenStreetMap can go a long way in developing their map database if they took on certain initiative to increase engagement with their service. One possible initiative would be for OpenStreetMap to form partnerships with educational institutions such as schools, or maybe libraries, to engage students with their service. As a way to develop computer and internet literacy, computer-related courses can teach students how to use OpenStreetMap. It'll expose them to online maps, GPS technology, how to participate in open source projects, and more - all while adding data to a free resource that could benefit the members of the community and the world.

Anticipated Problem: However, the concern here is that you might see an influx of dirty, unreliable data, particularly if the people behind them aren't very computer literate or only participating because it's a mandatory portion of a course. Naturally the data that come from volunteers who get involved because of their genuine passion for the project would be of higher quality.

Ensuring Data Consistency

For data improvement, the biggest problem I came across my data before I cleaned it was the lack of a unified format for street types or phone numbers, or simply incomplete information. If OpenStreetMap had a hard format that street types, phone numbers, zip codes, etc. should follow - and they ensured the format is appropriate for the city/country - there would be much cleaner data for analysis.

Anticipated Problem: The issue here is that you could very likely see a decrease in submissions due to the stricter guidelines.

Finally, open data are here to make average people's life better. For the non-data analyst, it would be nice if there was an application (mobile or web) that could evaluate the suitability of a potential rental house. The work addresses of all family members, importance weights on several amenities like supermarkets, convenience stores, cafes, public transportation, etc. and the application would calculate the suitability of each potential rental. The user would be able to sort them by score and compare them.

# Conclusion

It's clear from what we've seen that the mumbai OpenStretMap data is still incomplete and incorrect but there is still much in this city to be found and explored. The upside is that a lot of the data that has been entered is fairly clean, so future OSM users who embark on the task of improving the dataset with new information won't have much to worry about with regards to cleaning prior submissions. I'm looking forward to seeing how the data for the area grows overtime, and I hope to also play a role in that mission myself.